In [5]:
#Import Necessary Modules

import pandas as pd
import numpy as np
import wikipedia as wp
import geopy.geocoders
import requests
import io

#Lets encode the wikipedia page using wikipedia module

html = wp.page("List_of_postal_codes_of_Canada:_M").html().encode("UTF-8")

#Convert the following into DataFrame

data = pd.read_html(html, header=0)[0]
data.head()

data.drop(data[data["Borough"] == "Not assigned"].index, inplace=True)
data.head()

#If more than one neighborhood exists in one postal code area, then those two rows will combined into one row with the neighborhoods seperated by comma

data = data.groupby(['Postal Code', 'Borough'])['Neighbourhood'].apply(list).apply(lambda x: ', '.join(x)).to_frame().reset_index()
data.head()

#If a cell has a borough but a Not Assigned neighborhood, then the neighborhood will be same as the borough

for index, row in data.iterrows():
    if row["Neighbourhood"] == 'Not Assigned':
        row["Neighbourhood"] = row["Borough"]

data.shape

(103, 3)

In [11]:
#In order to use Foursquare location data, we need to get the latitude and longitude of corodinates of each neighborhood

url = "https://cocl.us/Geospatial_data"
s = requests.get(url).content
c = pd.read_csv(io.StringIO(s.decode('utf-8')))

In [12]:
c.columns = ["Postal Code", "Latitude", "Longitude"]
data = pd.merge(c, data, on="Postal Code")

In [13]:
data.head()

,Postal Code,Latitude,Longitude,Borough,Neighbourhood
0,M1B,43.806686,-79.194353,Scarborough,"Malvern, Rouge"
1,M1C,43.784535,-79.160497,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,43.763573,-79.188711,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,43.770992,-79.216917,Scarborough,Woburn
4,M1H,43.773136,-79.239476,Scarborough,Cedarbrae


In [14]:
data = data[["Postal Code", "Borough", "Neighbourhood", "Latitude", "Longitude"]]
data.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
